
# Coverage of Confidence Intervals: Exact vs. CLT (Large-Sample)

This notebook compares **empirical coverage** of confidence intervals built using:

- **Exact (true sampling distribution)**  
  - Bernoulli mean \(p\): **Clopper-Pearson** (Binomial inversion)  
  - Exponential rate \(\lambda\): **Chi-square / Gamma** (exact from \(\sum X_i\))

- **Large-sample CLT intervals**  
  - Bernoulli mean \(p\): Wald \( \hat p \pm z_{\alpha/2}\sqrt{\hat p(1-\hat p)/n}\)  
  - Exponential mean \(\mu\): \( \bar X \pm z_{\alpha/2}\, s/\sqrt{n}\) (and then transformed to \(\lambda=1/\mu\) if desired)

You can experiment with different sample sizes and see how coverage changes.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm, beta, chi2

rng = np.random.default_rng(12345)



## Helpers: Exact and CLT intervals


In [ ]:

def ci_bernoulli_clopper_pearson(x: int, n: int, alpha: float = 0.05):
    '''
    Exact (Clopper-Pearson) CI for Bernoulli p based on X ~ Binomial(n, p).
    Uses Beta quantiles.
    '''
    if n <= 0:
        raise ValueError("n must be positive")
    if not (0 <= x <= n):
        raise ValueError("x must be in {0,...,n}")

    lo = 0.0 if x == 0 else beta.ppf(alpha/2, x, n - x + 1)
    hi = 1.0 if x == n else beta.ppf(1 - alpha/2, x + 1, n - x)
    return lo, hi


def ci_bernoulli_wald(phat: float, n: int, alpha: float = 0.05):
    '''
    Large-sample (Wald) CI for p.
    '''
    z = norm.ppf(1 - alpha/2)
    se = np.sqrt(phat * (1 - phat) / n)
    return phat - z * se, phat + z * se


def ci_expon_rate_exact(sum_x: float, n: int, alpha: float = 0.05):
    '''
    Exact CI for exponential rate lambda when X_i iid Exp(rate=lambda).
    If S = sum X_i, then 2*lambda*S ~ chi2(df=2n).

      lambda_L = chi2.ppf(alpha/2, 2n) / (2S)
      lambda_U = chi2.ppf(1-alpha/2, 2n) / (2S)
    '''
    if sum_x <= 0:
        return (np.nan, np.nan)
    df = 2 * n
    lo = chi2.ppf(alpha/2, df) / (2 * sum_x)
    hi = chi2.ppf(1 - alpha/2, df) / (2 * sum_x)
    return lo, hi


def ci_expon_mean_clt(xbar: float, s: float, n: int, alpha: float = 0.05):
    '''
    CLT CI for the mean mu = E[X] using sample SD s:
      xbar +- z * s/sqrt(n)
    '''
    z = norm.ppf(1 - alpha/2)
    se = s / np.sqrt(n)
    return xbar - z * se, xbar + z * se



## Simulation engine

We compute coverage for each method by Monte Carlo simulation.

- For Bernoulli: generate n Bernoulli trials, compute p-hat, and build both intervals.
- For Exponential: generate n Exp(lambda) samples, compute S=sum X_i, x-bar, and sample SD s.


In [ ]:

def coverage_bernoulli(p: float, n: int, reps: int = 50_000, alpha: float = 0.05, seed: int = 12345):
    rng_local = np.random.default_rng(seed)
    x = rng_local.binomial(n, p, size=reps)
    phat = x / n

    # Exact
    exact_contains = np.empty(reps, dtype=bool)
    for i in range(reps):
        lo, hi = ci_bernoulli_clopper_pearson(int(x[i]), n, alpha)
        exact_contains[i] = (lo <= p <= hi)

    # CLT (Wald)
    z = norm.ppf(1 - alpha/2)
    se = np.sqrt(phat * (1 - phat) / n)
    lo = phat - z * se
    hi = phat + z * se
    wald_contains = (lo <= p) & (p <= hi)

    # Average interval lengths (exact length approximated by subsampling)
    m = min(5000, reps)
    exact_lengths = np.empty(m)
    for i in range(m):
        a, b = ci_bernoulli_clopper_pearson(int(x[i]), n, alpha)
        exact_lengths[i] = b - a

    return {
        "n": n,
        "p": p,
        "reps": reps,
        "alpha": alpha,
        "coverage_exact": float(exact_contains.mean()),
        "coverage_clt_wald": float(wald_contains.mean()),
        "avg_length_exact": float(exact_lengths.mean()),
        "avg_length_clt_wald": float(np.mean(hi - lo)),
    }


def coverage_exponential(lam: float, n: int, reps: int = 50_000, alpha: float = 0.05, seed: int = 12345):
    '''
    Compare:
      - Exact CI for rate lambda (chi-square pivot)
      - CLT CI for mean mu, then inverted to a CI for lambda via lambda=1/mu
    '''
    rng_local = np.random.default_rng(seed)
    x = rng_local.exponential(scale=1/lam, size=(reps, n))
    s = x.sum(axis=1)
    xbar = s / n
    sd = x.std(axis=1, ddof=1)

    # Exact rate CI
    lo_e = np.empty(reps)
    hi_e = np.empty(reps)
    for i in range(reps):
        lo_e[i], hi_e[i] = ci_expon_rate_exact(float(s[i]), n, alpha)
    exact_contains = (lo_e <= lam) & (lam <= hi_e)

    # CLT mean CI, then invert to lambda CI
    z = norm.ppf(1 - alpha/2)
    se = sd / np.sqrt(n)
    lo_m = xbar - z * se
    hi_m = xbar + z * se

    mu = 1/lam
    clt_mu_contains = (lo_m <= mu) & (mu <= hi_m)

    lo_lam = np.empty(reps)
    hi_lam = np.empty(reps)
    crosses = lo_m <= 0
    lo_lam[crosses] = 0.0
    hi_lam[crosses] = np.inf

    good = ~crosses
    lo_lam[good] = 1.0 / hi_m[good]
    hi_lam[good] = 1.0 / lo_m[good]
    clt_lam_contains = (lo_lam <= lam) & (lam <= hi_lam)

    finite = np.isfinite(hi_lam)
    avg_len_clt = float(np.mean(hi_lam[finite] - lo_lam[finite])) if finite.any() else float("inf")

    return {
        "n": n,
        "lambda": lam,
        "mu": 1/lam,
        "reps": reps,
        "alpha": alpha,
        "coverage_exact_lambda": float(exact_contains.mean()),
        "coverage_clt_mu": float(clt_mu_contains.mean()),
        "coverage_clt_lambda_inverted": float(clt_lam_contains.mean()),
        "avg_length_exact_lambda": float(np.mean(hi_e - lo_e)),
        "avg_length_clt_lambda_inverted_finite": avg_len_clt,
        "frac_clt_mu_interval_crosses_0": float(crosses.mean()),
    }



## Single experiment (quick)

Change the parameters below, run the cell, and inspect coverage.


In [ ]:

# --- Bernoulli example ---
p = 0.30
n = 20
reps = 50_000
alpha = 0.05

res_b = coverage_bernoulli(p=p, n=n, reps=reps, alpha=alpha, seed=2026)
pd.DataFrame([res_b])


In [ ]:

# --- Exponential example ---
lam = 2.0
n = 10
reps = 50_000
alpha = 0.05

res_e = coverage_exponential(lam=lam, n=n, reps=reps, alpha=alpha, seed=2026)
pd.DataFrame([res_e])



## Coverage vs sample size

Pick a grid of sample sizes and compare methods on one plot.


In [ ]:

def bernoulli_grid(p: float, ns, reps: int = 30_000, alpha: float = 0.05, seed: int = 12345):
    rows = []
    for i, n in enumerate(ns):
        rows.append(coverage_bernoulli(p=p, n=int(n), reps=reps, alpha=alpha, seed=seed + i))
    return pd.DataFrame(rows)

def exponential_grid(lam: float, ns, reps: int = 30_000, alpha: float = 0.05, seed: int = 12345):
    rows = []
    for i, n in enumerate(ns):
        rows.append(coverage_exponential(lam=lam, n=int(n), reps=reps, alpha=alpha, seed=seed + i))
    return pd.DataFrame(rows)


In [ ]:

# Bernoulli grid
p = 0.30
ns = [5, 10, 15, 20, 30, 50, 100, 200]
reps = 30_000
alpha = 0.05

dfb = bernoulli_grid(p=p, ns=ns, reps=reps, alpha=alpha, seed=2026)
dfb


In [ ]:

plt.figure()
plt.plot(dfb["n"], dfb["coverage_exact"], marker="o", label="Exact (Clopper-Pearson)")
plt.plot(dfb["n"], dfb["coverage_clt_wald"], marker="o", label="CLT (Wald)")
plt.axhline(1 - dfb["alpha"].iloc[0], linestyle="--", label="Nominal")
plt.xscale("log")
plt.ylim(0, 1)
plt.xlabel("n (log scale)")
plt.ylabel("Empirical coverage")
plt.title(f"Bernoulli(p={p}): coverage vs n (reps={reps})")
plt.legend()
plt.show()


In [ ]:

# Exponential grid
lam = 2.0
ns = [2, 3, 5, 10, 20, 30, 50, 100]
reps = 30_000
alpha = 0.05

dfe = exponential_grid(lam=lam, ns=ns, reps=reps, alpha=alpha, seed=2026)
dfe


In [ ]:

plt.figure()
plt.plot(dfe["n"], dfe["coverage_exact_lambda"], marker="o", label="Exact CI for lambda (chi-square)")
plt.plot(dfe["n"], dfe["coverage_clt_mu"], marker="o", label="CLT CI for mu (xbar +- z s/sqrt(n))")
plt.plot(dfe["n"], dfe["coverage_clt_lambda_inverted"], marker="o", label="CLT CI for lambda via inversion")
plt.axhline(1 - dfe["alpha"].iloc[0], linestyle="--", label="Nominal")
plt.xscale("log")
plt.ylim(0, 1)
plt.xlabel("n (log scale)")
plt.ylabel("Empirical coverage")
plt.title(f"Exponential(lambda={lam}): coverage vs n (reps={reps})")
plt.legend()
plt.show()



## Optional: explore different true parameters

Scan over a parameter grid at a fixed n.

Example: Bernoulli p in {0.05, 0.10, ..., 0.95} at one n.


In [ ]:

def bernoulli_param_scan(ps, n: int, reps: int = 30_000, alpha: float = 0.05, seed: int = 12345):
    rows = []
    for i, p in enumerate(ps):
        rows.append(coverage_bernoulli(p=float(p), n=n, reps=reps, alpha=alpha, seed=seed + i))
    return pd.DataFrame(rows)

ps = np.linspace(0.05, 0.95, 19)
n = 20
reps = 30_000
alpha = 0.05

df_scan = bernoulli_param_scan(ps=ps, n=n, reps=reps, alpha=alpha, seed=2026)
df_scan.head()


In [ ]:

plt.figure()
plt.plot(df_scan["p"], df_scan["coverage_exact"], marker="o", label="Exact (Clopper-Pearson)")
plt.plot(df_scan["p"], df_scan["coverage_clt_wald"], marker="o", label="CLT (Wald)")
plt.axhline(1 - df_scan["alpha"].iloc[0], linestyle="--", label="Nominal")
plt.ylim(0, 1)
plt.xlabel("True p")
plt.ylabel("Empirical coverage")
plt.title(f"Bernoulli: coverage vs p at n={n} (reps={reps})")
plt.legend()
plt.show()



## Notes / talking points

- Exact methods are conservative in small samples (coverage often above nominal).
- Wald for Bernoulli can undercover badly when p is near 0 or 1, or when n is small.
- For exponential data, the exact rate CI uses a chi-square pivot.
- CLT CI for mu is often reasonable once n is moderate, but converting it to a rate CI by inversion can behave oddly if the mean CI crosses 0.
